In [5]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import matplotlib.pyplot as plt
from pandas import DataFrame
import numpy as np

In [7]:
# DB.load_data()

In [8]:
from ICARUS.Database import DB2D
from ICARUS.Database import DB
from ICARUS.Airfoils.airfoil import Airfoil
import os

fig, axs = plt.subplots(3,1)
airf_root = DB.get_airfoil("S4062")
airf_tip = DB.get_airfoil("NACA2412")


ax = axs[0]
ax2 = axs[1]
ax3 = axs[2]

ax.set_title("Root")
airf_root.plot(scatter= True, ax = ax)
airf_root.repanel_spl(200)
airf_root.plot(ax = ax)

ax2.set_title("Tip")
airf_tip.plot(scatter= True, ax = ax2)
# airf_tip.repanel_spl(200)
airf_tip.plot(ax = ax2)

ax3.set_title("Morphed")
airf = Airfoil.morph_new_from_two_foils(
    airf_root, airf_tip, 0.0, 100
)

airf.plot(ax = ax3)
airf.repanel_spl(200)
airf.plot(camber = True, scatter = True, ax = ax3)

In [9]:
airf = Airfoil.morph_new_from_two_foils(
    airf_root, airf_tip, 1., 100
)
fig, ax = plt.subplots()
airf.plot(ax = ax)
# for i in range(100):
airf.repanel_spl(200, smoothing=1e-4)
airf.plot(camber = True, scatter = True, ax = ax )

In [10]:
from xfoil import XFoil
from xfoil.model import Airfoil as XFAirfoil

xf = XFoil()
xf.Re = 1e6
xf.M = 0.0
pts = airf.selig
xpts = pts[0]
ypts = pts[1]
xf_airf_obj = XFAirfoil(x=xpts, y=ypts)
xf.airfoil = xf_airf_obj
xf.max_iter = 500
xf.xtr = (1., 1.)
xf.n_crit = 9
xf.print = True
xf.repanel(120)
xf.filter()


aXF, clXF, cdXF, cmXF, cpXF = xf.aseq(-10, 15, 0.25)
dat = np.array([aXF, clXF, cdXF, cmXF], dtype=float).T
df: DataFrame = DataFrame(dat)

# Rename columns
df.columns = ["AoA", "CL", "CD", "Cm"]
df.set_index("AoA", inplace=True)


# Plor CL/CD
fig, ax = plt.subplots(2, 2)
aoa = df.index
df["CL/CD"] = df["CL"] / df["CD"]

ax[0,0].plot(aoa, df["CL/CD"], label="CL/CD")
ax[0,0].set_xlabel("AoA")
ax[0,0].set_ylabel("CL/CD")

ax[0,1].plot(aoa, df["CL"], label="CL")
ax[0,1].set_xlabel("AoA")
ax[0,1].set_ylabel("CL")

ax[1,0].plot(aoa, df["CD"], label="CD")
ax[1,0].set_xlabel("AoA")
ax[1,0].set_ylabel("CD")

ax[1,1].plot(aoa, df["Cm"], label="Cm")
ax[1,1].set_xlabel("AoA")
ax[1,1].set_ylabel("Cm")

Text(0, 0.5, 'Cm')